In [1]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/properties-for-sale-in-the-south-east-of-england/Rightmove_South-Eastern_Station_Properties_2022-08-01_13-34-18.csv


In [2]:
data = pd.read_csv("/kaggle/input/properties-for-sale-in-the-south-east-of-england/Rightmove_South-Eastern_Station_Properties_2022-08-01_13-34-18.csv")

In [3]:
data.head()

,Station,Price,Property_Size,Address,Distance from Station (miles),Description,Date Added,Added By Whom,Telephone Number,Commute Time
0,Abbey Wood station,£100000,2 bedroom flat for sale,7 Avocet Mews Woolwich London SE28 0DA,1.55,**For Sale By Public Auction 3rd August 2022 0...,Added on 13/07/2022,by Auction House London Hampstead,020 8012 3603,31
1,Abbey Wood station,£100000,2 bedroom flat for sale,16 Mossdown Close Belvedere Kent DA17 5QW,1.32,**For Sale By Public Auction 3rd August 2022 0...,Added on 31/03/2022,by Auction House London Hampstead,020 8012 3603,31
2,Abbey Wood station,£120000,1 bedroom flat for sale,Flat Park Crescent Erith,2.20,100 Key Properties is pleased to present this ...,Reduced on 21/07/2022,by 100 Key Properties Hornchurch,020 8012 3340,31
3,Abbey Wood station,£125000,1 bedroom flat for sale,Flat 323 The Vista Building 30 Calderwood Stre...,2.42,**For Sale By Public Auction 3rd August 2022 0...,Added on 13/07/2022,by Auction House London Hampstead,020 8012 3603,31
4,Abbey Wood station,£125000,3 bedroom flat for sale,32 Brinkburn Close Abbey Wood London SE2 9EJ,0.60,**For Sale By Public Auction 3rd August 2022 0...,Added on 12/07/2022,by Auction House London Hampstead,020 8012 3603,31


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41873 entries, 0 to 41872
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Station                        41873 non-null  object 
 1   Price                          41873 non-null  object 
 2   Property_Size                  41873 non-null  object 
 3   Address                        41873 non-null  object 
 4   Distance from Station (miles)  41873 non-null  float64
 5   Description                    41847 non-null  object 
 6   Date Added                     41873 non-null  object 
 7   Added By Whom                  41873 non-null  object 
 8    Telephone Number              41873 non-null  object 
 9   Commute Time                   41873 non-null  int64  
dtypes: float64(1), int64(1), object(8)
memory usage: 3.2+ MB


Let's clean the Price column. 

In [5]:
data.Price = data.Price.str.replace("£","")
data.Price = data.Price.str.replace("POA","")
data.Price = pd.to_numeric(data.Price, errors='coerce')

Next, let's extract the dates from the Date Added (string) column. After some research and testing, I'd use the following approach.

Please feel free to suggest a faster and more elegant method. 

In [6]:
pip install datefinder

Note: you may need to restart the kernel to use updated packages.


In [7]:
import datefinder
data["Date Added New"] = data["Date Added"].apply(lambda x: list(datefinder.find_dates(x)))
data["Date Added New"] = data["Date Added New"].apply(lambda x: ' '.join(map(str, x)))
data["Date Added New"] = pd.to_datetime(data["Date Added New"], format = "%Y-%m-%d %H:%M:%S")
data = data.drop(["Date Added"],axis = 1)

In [8]:
data.head()

,Station,Price,Property_Size,Address,Distance from Station (miles),Description,Added By Whom,Telephone Number,Commute Time,Date Added New
0,Abbey Wood station,100000.0,2 bedroom flat for sale,7 Avocet Mews Woolwich London SE28 0DA,1.55,**For Sale By Public Auction 3rd August 2022 0...,by Auction House London Hampstead,020 8012 3603,31,2022-07-13
1,Abbey Wood station,100000.0,2 bedroom flat for sale,16 Mossdown Close Belvedere Kent DA17 5QW,1.32,**For Sale By Public Auction 3rd August 2022 0...,by Auction House London Hampstead,020 8012 3603,31,2022-03-31
2,Abbey Wood station,120000.0,1 bedroom flat for sale,Flat Park Crescent Erith,2.20,100 Key Properties is pleased to present this ...,by 100 Key Properties Hornchurch,020 8012 3340,31,2022-07-21
3,Abbey Wood station,125000.0,1 bedroom flat for sale,Flat 323 The Vista Building 30 Calderwood Stre...,2.42,**For Sale By Public Auction 3rd August 2022 0...,by Auction House London Hampstead,020 8012 3603,31,2022-07-13
4,Abbey Wood station,125000.0,3 bedroom flat for sale,32 Brinkburn Close Abbey Wood London SE2 9EJ,0.60,**For Sale By Public Auction 3rd August 2022 0...,by Auction House London Hampstead,020 8012 3603,31,2022-12-07


I don't need the following three columns. Perhaps I would've used some information from the Description col for EDA purposes (ie, For Sale vs Relisted etc) 

In [9]:
data = data.drop(["Description", "Added By Whom", " Telephone Number"], axis = 1)

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41873 entries, 0 to 41872
Data columns (total 7 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Station                        41873 non-null  object        
 1   Price                          41860 non-null  float64       
 2   Property_Size                  41873 non-null  object        
 3   Address                        41873 non-null  object        
 4   Distance from Station (miles)  41873 non-null  float64       
 5   Commute Time                   41873 non-null  int64         
 6   Date Added New                 41350 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 2.2+ MB


Let's extract the UK Postcodes from the Address column.

Don't forget to replace the empty spaces '' with a nan and drop them afterwords.

In [11]:
data["post_codes"] = data.Address.str.findall(r"[A-Z]{1,2}[0-9R][0-9A-Z]? [0-9][A-Z]{2}")
data.post_codes = data.post_codes.apply(lambda x: ' '.join(map(str, x)))

data1 = data.replace(r'^\s*$', np.nan, regex=True)

In [12]:
data1

,Station,Price,Property_Size,Address,Distance from Station (miles),Commute Time,Date Added New,post_codes
0,Abbey Wood station,100000.0,2 bedroom flat for sale,7 Avocet Mews Woolwich London SE28 0DA,1.55,31,2022-07-13,SE28 0DA
1,Abbey Wood station,100000.0,2 bedroom flat for sale,16 Mossdown Close Belvedere Kent DA17 5QW,1.32,31,2022-03-31,DA17 5QW
2,Abbey Wood station,120000.0,1 bedroom flat for sale,Flat Park Crescent Erith,2.20,31,2022-07-21,NaN
3,Abbey Wood station,125000.0,1 bedroom flat for sale,Flat 323 The Vista Building 30 Calderwood Stre...,2.42,31,2022-07-13,SE18 6JF
4,Abbey Wood station,125000.0,3 bedroom flat for sale,32 Brinkburn Close Abbey Wood London SE2 9EJ,0.60,31,2022-12-07,SE2 9EJ
...,...,...,...,...,...,...,...,...
41868,Wye station,330000.0,3 bedroom house for sale,Larch Walk Kennington Ashford,2.54,84,2022-07-28,NaN
41869,Wye station,335000.0,3 bedroom semi-detached house for sale,Rylands Road Kennington Ashford,2.60,84,2022-07-15,NaN
41870,Wye station,350000.0,3 bedroom apartment for sale,Harville Road ASHFORD,0.48,84,2022-05-13,NaN
41871,Wye station,350000.0,3 bedroom detached house for sale,Jersey Close Kennington Ashford Kent TN24,2.21,84,2022-06-25,NaN


In [13]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41873 entries, 0 to 41872
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Station                        41873 non-null  object        
 1   Price                          41860 non-null  float64       
 2   Property_Size                  41873 non-null  object        
 3   Address                        41873 non-null  object        
 4   Distance from Station (miles)  41873 non-null  float64       
 5   Commute Time                   41873 non-null  int64         
 6   Date Added New                 41350 non-null  datetime64[ns]
 7   post_codes                     2339 non-null   object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 2.6+ MB


Oh that's a bit of disappointment. There are only 2339 post codes! I don't need them then. 

That's a problem that Rightmove should fix!!!

In [14]:
data = data.drop(["Address","post_codes"], axis =1)

Let's transform the long strings into integers. 

In [15]:
from sklearn.preprocessing import LabelEncoder
LB = LabelEncoder()
data["property_encoded"] = LB.fit_transform(data["Property_Size"])

print(data.property_encoded.nunique())
print(data.Property_Size.nunique())

107
107


In [16]:
data["station_encoded"] = LB.fit_transform(data["Station"])
data = data.drop(["Station","Property_Size"], axis = 1)

In [17]:
data = data.dropna()
data.head()

,Price,Distance from Station (miles),Commute Time,Date Added New,property_encoded,station_encoded
0,100000.0,1.55,31,2022-07-13,33,0
1,100000.0,1.32,31,2022-03-31,33,0
2,120000.0,2.20,31,2022-07-21,11,0
3,125000.0,2.42,31,2022-07-13,11,0
4,125000.0,0.60,31,2022-12-07,56,0


In [18]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn import metrics  

y = data['Price']
X = data[["Distance from Station (miles)", "Commute Time","property_encoded"]]
#X = data['Distance from Station (miles)'].values.reshape(-1,1)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.85)

regressor = LinearRegression()  
regressor.fit(X_train, y_train) 

y_pred = regressor.predict(X_train) 
print("R2_score: " ,r2_score(y_train, y_pred))
print('Mean Absolute Error (Train set):',metrics.mean_absolute_error(y_train, y_pred)) 
rmse = np.sqrt(np.mean((y_train - y_pred)**2))
print('Root Mean Absolute Error (Train set):',rmse)

R2_score:  0.08157046707902205
Mean Absolute Error (Train set): 42894.01472551705
Root Mean Absolute Error (Train set): 53701.626327430684


In [19]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error
from math import sqrt
y = data['Price']
X = data.drop(['Price','Date Added New'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.85)
knn_model = KNeighborsClassifier(n_neighbors=1)
knn_model.fit(X_train, y_train)

train_preds = knn_model.predict(X_train)
mse = mean_squared_error(y_train, train_preds)
rmse = sqrt(mse)
print('Root Mean Squared Error (Train set):',rmse)
knn_model.score(X_train,y_train)

Root Mean Squared Error (Train set): 23740.44289543448


0.8332764116575592

In [20]:
test_preds = knn_model.predict(X_test)
mse = mean_squared_error(y_test, test_preds)
rmse = sqrt(mse)
print('Root Mean Squared Error (Test set):',rmse)
knn_model.score(X_test,y_test)

Root Mean Squared Error (Test set): 56105.74251261732


0.12385099177552007

Wow this is unsatisfactory! I will work on my local machine to achieve a better result. 